#### Bibliotecas

In [7]:
library(microdatasus)
library(stringr)
library(dplyr)
library(magrittr)
library(tidyr)

#### Configurações de tela

In [8]:
options(OutDec = ",")
options(digits = 4, scipen = 100)

#### 1 - Dados de 2019

#### 1.1 - Março de 2019

#### Importando a base relativo aos profissionais

In [ ]:
#Ano 2019 - Março:
mar_19 <- fetch_datasus(month_start = 3, month_end = 3,
                        year_start = 2019, year_end = 2019,
                        uf = "PB", information_system = "CNES-PF")

In [10]:
#Processando os dados: nomeando as variáveis
mar_19 <- process_cnes(mar_19, information_system = "CNES-PF")

#### Descrição da base de dados

In [11]:
glimpse(mar_19)

Rows: 68.610
Columns: 49
$ CNES         <chr> "2321742", "2321742", "2321742", "2321742", "2321742", "2~
$ CODUFMUN     <chr> "250010", "250010", "250010", "250010", "250010", "250010~
$ REGSAUDE     <chr> "0011", "0011", "0011", "0011", "0011", "0011", "0011", "~
$ MICR_REG     <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N~
$ DISTRSAN     <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N~
$ DISTRADM     <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N~
$ TPGESTAO     <chr> "Municipal", "Municipal", "Municipal", "Municipal", "Muni~
$ PF_PJ        <chr> "Pessoa jurídica", "Pessoa jurídica", "Pessoa jurídica", ~
$ CPF_CNPJ     <chr> "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1", "1~
$ NIV_DEP      <chr> "Mantida", "Mantida", "Mantida", "Mantida", "Mantida", "M~
$ CNPJ_MAN     <chr> "226", "226", "226", "226", "226", "226", "226", "226", "~
$ ESFERA_A     <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N~
$ ATIVIDAD     

#### Selecionando algumas variáveis

In [12]:
dados_mar_19 = mar_19 %<>%
  select(CNES, CNS_PROF, CODUFMUN, TP_UNID, CBO, HORAOUTR, HORAHOSP,
         HORA_AMB, NAT_JUR)

#### Filtrando informações


In [13]:
#Filtrando os tipos de unidades:
dados_mar_19 <- mar_19 %>%
  filter(str_detect(TP_UNID, "Hospital|Pronto"))

#Filtrando apenas os profissionais da saúde:
dados_mar_19 %<>%
  mutate(CBO = as.character(CBO)) %>%
  filter(str_starts(CBO, "223") | str_starts(CBO, "225") |
         str_starts(CBO, "322"))

#### Descobrindo o nome das profissões 

In [14]:
cbo = read.csv2("CBO2002 - Ocupacao.csv") %>%
  mutate(CBO = as.character(CODIGO)) %>%
  rename(PROFISSAO = TITULO) %>%
  select(-CODIGO)

dados_mar_19 = left_join(dados_mar_19, cbo, by = "CBO")

In [15]:
#Atentando aos NA's
dados_mar_19 %>% summarise_all(~ sum(is.na(.)))

#Descobrindo quais cbo não foram entendidos
ausencia_cbo = dados_mar_19 %>% 
  filter(!complete.cases(PROFISSAO))

#Confirmando se todos os códigos iniciam com 223
ausencia_cbo %>% summarise(n = sum(str_starts(CBO, "223"), na.rm = TRUE))


CNES,CNS_PROF,CODUFMUN,TP_UNID,CBO,HORAOUTR,HORAHOSP,HORA_AMB,NAT_JUR,PROFISSAO
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,0,0,0,0,0,211


n
<int>
211


In [16]:
#Tendo em vista que 223 corresponde ao grupo dos médicos, onde é 
#NA substituir por Médico
dados_mar_19 %<>%
  mutate_at(vars(PROFISSAO), ~replace_na(., "Médico"))

#### Identificando o profissional por estabelecimento 

In [17]:
dados_mar_19 %<>%
  group_by(CNS_PROF, CNES) %>%
  mutate(id = row_number()) %>%
  ungroup()

#### Importando dados relativos a quantidade de leitos no hospital

In [ ]:
leitos = fetch_datasus(month_start = 3, month_end = 3,
                        year_start = 2019, year_end = 2019,
                        uf = "PB", information_system = "CNES-LT")

In [19]:
leitos %<>%
  group_by(CNES) %>%
  summarise(TOTAL = sum(QT_EXIST))

#### Unindo as duas bases anteriores

In [20]:
dados_mar_19 <- left_join(dados_mar_19, leitos, by = "CNES")

A não correspondência entre *dados_mar_19* e *leitos*  corresponde ao fato de não ter pleito no hospital. 
Assim, substitui o NA por zero. 

In [21]:
dados_mar_19 %<>%
  mutate_at(vars(TOTAL), ~replace_na(., 0))

#### 1.2 - Dezembro de 2019

In [ ]:
#Ano 2019 - Dezembro
dez_19 = fetch_datasus(month_start = 12, month_end = 12,
                        year_start = 2019, year_end = 2019,
                        uf = "PB", information_system = "CNES-PF")

In [23]:
#Processando os dados: nomeando as variáveis
dez_19 = process_cnes(dez_19, information_system = "CNES-PF")

#### Selecionado variáveis

In [24]:
dados_dez_19 = dez_19 %<>%
  select(CNES, CNS_PROF, CODUFMUN, TP_UNID, CBO, HORAOUTR, HORAHOSP,
         HORA_AMB, NAT_JUR)

#### Filtrando informações

In [25]:
#Filtrando os tipos de unidades:
dados_dez_19 %<>%
  filter(str_detect(TP_UNID, "Hospital|Pronto"))

#Filtrando apenas os profissionais da saúde
dados_dez_19 %<>%
  mutate(CBO = as.character(CBO)) %>%
  filter(str_starts(CBO, "223") | str_starts(CBO, "225") |
         str_starts(CBO, "322"))


#### Descobrindo o nome da profissão

In [26]:
dados_dez_19 = left_join(dados_dez_19, cbo, by = "CBO")

In [27]:
#Atentando aos NA's
dados_dez_19 %>% summarise_all(~ sum(is.na(.)))

#Descobrindo quais cbo não foram entendidos
ausencia_cbo = dados_dez_19  %>% 
  filter(!complete.cases(PROFISSAO))

#Confirmando se todos os códigos iniciam com 223
ausencia_cbo %>% summarise(n = sum(str_starts(CBO, "223"), na.rm = TRUE))


CNES,CNS_PROF,CODUFMUN,TP_UNID,CBO,HORAOUTR,HORAHOSP,HORA_AMB,NAT_JUR,PROFISSAO
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,0,0,0,0,0,271


n
<int>
271


In [28]:
#Tendo em vista que 223 corresponde ao grupo dos médicos, onde é NA substituir por Médico
dados_dez_19  %<>%
  mutate_at(vars(PROFISSAO), ~replace_na(., "Médico"))

#### Identificando o profissional por estabelecimento 

In [29]:
dados_dez_19 %<>%
  group_by(CNS_PROF, CNES) %>%
  mutate(id = row_number()) %>%
  ungroup

Deixando apenas as informações em dezembro que tem em março. Para isso, utiliza-se o `semi-join` que retorna todas as linhas de onde há valores correspondentes de y, mantendo apenas as colunas de x: 

In [30]:
dados_dez_19 = semi_join(dados_dez_19, dados_mar_19, by = "CNES")

#### Importando a base relativo a quantidade de leitos

In [ ]:
leitos = fetch_datasus(month_start = 12, month_end = 12,
                        year_start = 2019, year_end = 2019,
                        uf = "PB", information_system = "CNES-LT")

#### Tratando os dados conforme anteriormente

In [32]:
#Contando a quantidade de leitos por CNES
leitos %<>%
  group_by(CNES) %>%
  summarise(TOTAL = sum(QT_EXIST))

#Unindo a base leitos com dados_dez_19
dados_dez_19 <- left_join(dados_dez_19, leitos, by = "CNES")

#Onde tem NA substitui por 0
dados_dez_19 %<>%
  mutate_at(vars(TOTAL), ~replace_na(., 0))

#### 1.3 - Março e Dezembro de 2019

O objetivo dessa união é descobrir qual profissional rotacionou entre março e dezembro

In [33]:
#Preparando a base - março
d0 <-  dados_mar_19 %>%
  select(CNES, CNS_PROF) %>%
  group_by(CNS_PROF, CNES) %>%
  mutate(t0 = 1,
         id = row_number()) %>%
  ungroup()

#Preparando a base - dezembro
d1 <-  dados_dez_19 %>%
  select(CNES, CNS_PROF) %>%
  group_by(CNS_PROF, CNES) %>%
  mutate(t1 = 1,
         id = row_number()) %>%
  ungroup()

#Unindo as informaçoes dos dois meses:
d <- full_join(d0, d1, by = c("CNES", "CNS_PROF", "id"))

#Onde nao tem informaçao, substituir por 0
d[is.na(d)] = 0

#Se trabalhou nos dois periodos continuou no emprego, menor do que 2 trocou:
d = d %>%
  mutate(t = t0 + t1,
         rotat = if_else(t < 2, 1,0))

#Voltando para a base original:
df0 = left_join(dados_mar_19,d, by = c("CNES", "CNS_PROF", "id")) %>%
  mutate(tempo = 0)

df1 = left_join(dados_dez_19,d, by = c("CNES", "CNS_PROF", "id")) %>%
  mutate(tempo = 1)

#Uniao dos dois meses:
uniao_19 = bind_rows(df0, df1)

####  Importando dados sobre as mesoregiões

In [34]:
codigo <- read.csv2("codigo_meso.csv", sep = ";")

In [35]:
#Transformando a tipagem da variável CODUFMUN
codigo %<>%
    mutate(CODUFMUN = as.character(mun_trab)) %>%
    select(-mun_trab)

#Unindo a base uniao_19 com codigo
uniao_19 = inner_join(uniao_19, codigo, by = "CODUFMUN")

#### Criando atributos sobre os hospitais e profissionais


In [36]:
#Quantidade de vínculos em março e dezembro
uniao_19 %<>%
  group_by(CNS_PROF, tempo) %>%
  mutate(VINCULOS = n())

In [37]:
uniao_19 %<>%
  mutate(NAT_JUR = as.character(NAT_JUR),
         PUBLICO = if_else(NAT_JUR == "2011" |
                          NAT_JUR == "Autarquia Federal" |
                          NAT_JUR == "Autarquia Municipal" |
                          NAT_JUR == "Município" |
                          NAT_JUR == "Órgão Público do Poder Executivo Estadual ou do Distrito Federal", 1, 0))

In [38]:
#Transformando subgrupo de profissões em uma geral
uniao_19  %<>%
  mutate(PROFISSAO = case_when(str_detect(PROFISSAO,"Auxiliar de enfermagem") ~ "Técnico de enfermagem",
                               str_starts(PROFISSAO, "Médico") ~ "Médico",
                               str_starts(PROFISSAO, "Fisioterapeuta") ~ "Fisioterapeuta",
                               str_starts(PROFISSAO, "Farmacêutico") ~ "Farmacêutico",
                               str_starts(PROFISSAO, "Enfermeiro") ~ "Enfermeiro",
                               TRUE ~ as.character(PROFISSAO)))

In [39]:
#Número de horas trabalhadas
uniao_19 %<>%
  mutate(HORAOUTR = as.numeric(HORAOUTR),
         HORAHOSP = as.numeric(HORAHOSP),
         HORA_AMB = as.numeric(HORA_AMB),
         H_TOTAL = HORAOUTR + HORAHOSP + HORA_AMB)

#Criando dummies para total de horas trabalhadas:
uniao_19 %<>% mutate(D_HORAS = case_when(H_TOTAL %in% 0:10 ~ 1,
                                         H_TOTAL %in% 11:20 ~ 2,
                                         H_TOTAL %in% 21:30 ~ 3,
                                         H_TOTAL > 30 ~ 4))

In [40]:
#Dummy para indicar o porte do hospital
uniao_19  %<>% mutate(PORTE = case_when(TOTAL <= 50 ~ 1,
                                        TOTAL %in% 51:150 ~ 2,
                                        TOTAL %in% 151:500 ~ 3,
                                        TOTAL > 500 ~ 4))

In [41]:
#Dummies para indicar a quantidade de vínculos trabalhistas
uniao_19  %<>% mutate(D_VINC = case_when(VINCULOS == 1 ~ "1 vínculo",
                                         VINCULOS == 2 ~ "2 vínculos",
                                         VINCULOS >= 3 ~ "3 ou mais vínculos"))

In [42]:
#Desagrupando os dados
uniao_19 %<>%
    ungroup()

In [43]:
glimpse(uniao_19)

Rows: 42.103
Columns: 24
$ CNES      <chr> "2322153", "2322153", "2322153", "2322153", "2322153", "2322~
$ CNS_PROF  <chr> "207428997690007", "980016288637438", "980016297404556", "98~
$ CODUFMUN  <chr> "250020", "250020", "250020", "250020", "250020", "250020", ~
$ TP_UNID   <chr> "Hospital geral", "Hospital geral", "Hospital geral", "Hospi~
$ CBO       <chr> "225124", "225125", "225125", "322205", "322205", "223605", ~
$ HORAOUTR  <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ~
$ HORAHOSP  <dbl> 0, 0, 12, 0, 0, 0, 12, 40, 0, 40, 0, 0, 12, 0, 12, 0, 0, 0, ~
$ HORA_AMB  <dbl> 8, 18, 12, 40, 40, 20, 12, 0, 20, 0, 24, 24, 12, 40, 12, 20,~
$ NAT_JUR   <chr> "Órgão Público do Poder Executivo Estadual ou do Distrito Fe~
$ PROFISSAO <chr> "Médico", "Médico", "Médico", "Técnico de enfermagem", "Técn~
$ id        <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ~
$ TOTAL     <int> 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, ~
$ t0        <db

#### 2 - Dados de 2020

Observação: tratamento de dados em 2020 é o mesmo realizado em 2019.

#### 2.1 Março de 2020

#### Importando os dados

In [ ]:
#Ano 2020 - Março:
mar_20 = fetch_datasus(month_start = 3, month_end = 3,
                        year_start = 2020, year_end = 2020,
                        uf = "PB", information_system = "CNES-PF")

#Processando os dados: nomeando as variaveis
mar_20 = process_cnes(mar_20, information_system = "CNES-PF")

#### Selecionando informações

In [45]:
dados_mar_20 = mar_20 %<>%
  select(CNES, CNS_PROF, CODUFMUN, TP_UNID, CBO, HORAOUTR, HORAHOSP,
         HORA_AMB, NAT_JUR)

#### Filtrando informações 

In [46]:
#Filtrando os tipos de unidades:
dados_mar_20 %<>%
  filter(str_detect(TP_UNID, "Hospital|Pronto"))

#Filtrando apenas os profissionais da saúde
dados_mar_20 %<>%
  mutate(CBO = as.character(CBO)) %>%
  filter(str_starts(CBO, "223") | str_starts(CBO, "225") |
         str_starts(CBO, "322"))

#### Descobrindo o nome das profissões 

In [47]:
dados_mar_20 = left_join(dados_mar_20, cbo, by = "CBO")

In [48]:
#Atentando aos NA's
dados_mar_20 %>% summarise_all(~ sum(is.na(.)))

#Descobrindo quais cbo não foram entendidos
ausencia_cbo = dados_mar_20  %>% 
  filter(!complete.cases(PROFISSAO))

#Confirmando se todos os códigos iniciam com 223
ausencia_cbo %>% summarise(n = sum(str_starts(CBO, "223"), na.rm = TRUE))

CNES,CNS_PROF,CODUFMUN,TP_UNID,CBO,HORAOUTR,HORAHOSP,HORA_AMB,NAT_JUR,PROFISSAO
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,0,0,0,0,0,282


n
<int>
282


In [49]:
#Substituindo NA substituir por Médico
dados_mar_20  %<>%
  mutate_at(vars(PROFISSAO), ~replace_na(., "Médico"))

#### Identificando os profissionais por CNES

In [50]:
#Identificando o profissional dentro do estabelecimento
dados_mar_20 %<>%
  group_by(CNS_PROF, CNES) %>%
  mutate(id = row_number()) %>%
  ungroup()

#Deixando apenas os hospitais que tem em 2019
dados_mar_20 <- semi_join(dados_mar_20, dados_mar_19, by = "CNES")

#### Importando e tratando dados relativos aos leitos

In [ ]:
#Importando dados
leitos <- fetch_datasus(month_start = 3, month_end = 3,
                        year_start = 2020, year_end = 2020,
                        uf = "PB", information_system = "CNES-LT")

#Somatório de leitos por hospital
leitos %<>%
  group_by(CNES) %>%
  summarise(TOTAL = sum(QT_EXIST))

#Unindo os dados de leitos com dados_mar_20
dados_mar_20 <- left_join(dados_mar_20, leitos, by = "CNES")

#Onde há NA susbtituir por zero
dados_mar_20 %<>%
  mutate_at(vars(TOTAL), ~replace_na(., 0))

#### 2.2 Dezembro de 2020

#### Importando os dados

In [ ]:
#Importando os dados
dez_20 = fetch_datasus(month_start = 12, month_end = 12,
                        year_start = 2020, year_end = 2020,
                        uf = "PB", information_system = "CNES-PF")

#Processando os dados: nomeando as variaveis
dez_20 = process_cnes(dez_20,  information_system = "CNES-PF")

#### Selecionando variáveis

In [53]:
dados_dez_20 = dez_20 %<>%
  select(CNES, CNS_PROF, CODUFMUN, TP_UNID, CBO, HORAOUTR, HORAHOSP,
         HORA_AMB, NAT_JUR)

#### Filtrando as informações

In [54]:
#Filtrando por tipo de unidade
dados_dez_20 %<>%
  filter(str_detect(TP_UNID, "Hospital|Pronto"))

#Filtrando apenas os profissionais da saúde
dados_dez_20 %<>%
  mutate(CBO = as.character(CBO)) %>%
  filter(str_starts(CBO, "223") | str_starts(CBO, "225") |
         str_starts(CBO, "322"))

#### Descobrindo o nome das profissões

In [55]:
#Unindo a base cbo com dados_dez_20
dados_dez_20 = left_join(dados_dez_20, cbo, by = "CBO")

#Atentando aos NA's
dados_dez_20 %>% summarise_all(~ sum(is.na(.)))

#Descobrindo quais cbo não foram entendidos
ausencia_cbo = dados_dez_20  %>% 
  filter(!complete.cases(PROFISSAO))

#Confirmando se todos os códigos iniciam com 223
ausencia_cbo %>% summarise(n = sum(str_starts(CBO, "223"), na.rm = TRUE))


CNES,CNS_PROF,CODUFMUN,TP_UNID,CBO,HORAOUTR,HORAHOSP,HORA_AMB,NAT_JUR,PROFISSAO
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,0,0,0,0,0,316


n
<int>
316


In [56]:
#Substituindo NA substituir por Médico
dados_dez_20  %<>%
  mutate_at(vars(PROFISSAO), ~replace_na(., "Médico"))

#### Identificando os profissionais

In [57]:
#Identificando o profissional dentro do estabelecimento
dados_dez_20 %<>%
  group_by(CNS_PROF, CNES) %>%
  mutate(id = row_number()) %>%
  ungroup()

#Deixando informaçoes em dezembro que tem em 2019 (mês de referência = mar/2019)
dados_dez_20 = semi_join(dados_dez_20, dados_mar_19, by = "CNES")


#### Importando e tratando dados relativos aos leitos

In [ ]:
#Leitos
leitos = fetch_datasus(month_start = 12, month_end = 12,
                        year_start = 2020, year_end = 2020,
                        uf = "PB", information_system = "CNES-LT")
#Somatório
leitos %<>%
  group_by(CNES) %>%
  summarise(TOTAL = sum(QT_EXIST))

#União das bases
dados_dez_20 = left_join(dados_dez_20, leitos, by = "CNES")

#Substituir NA por zero
dados_dez_20 %<>%
  mutate_at(vars(TOTAL), ~replace_na(., 0))

#### 2.3 Março e Dezembro de 2020

In [59]:
#Preparando a base - março
d0 =  dados_mar_20 %>%
  select(CNES, CNS_PROF) %>%
  group_by(CNS_PROF, CNES) %>%
  mutate(t0 = 1,
         id = row_number()) %>%
  ungroup()

#Preparando a base - dezembro
d1 =  dados_dez_20 %>%
  select(CNES, CNS_PROF) %>%
  group_by(CNS_PROF, CNES) %>%
  mutate(t1 = 1,
         id = row_number()) %>%
  ungroup()

#Unindo as informaçoes dos dois meses:
d = full_join(d0, d1, by = c("CNES", "CNS_PROF", "id"))

#Onde nao tem informaçao, substituir por 0
d[is.na(d)] = 0

#Se trabalhou nos dois periodos continuou no emprego, menor do que 2 trocou:
d = d %>% 
  mutate(t = t0 + t1,
         rotat = if_else(t < 2, 1,0))

#Voltando para a base original:
df0 = left_join(dados_mar_20, d, by = c("CNES", "CNS_PROF", "id")) %>%
  mutate(tempo = 0)

df1 = left_join(dados_dez_20, d, by = c("CNES", "CNS_PROF", "id")) %>%
  mutate(tempo = 1)

#Uniao dos dois meses:
uniao_20 = bind_rows(df0, df1)

#### Criando atributos relativos aos hospitais e profissionais

In [60]:
#Quantidade de vínculos entre março e dezembro.
uniao_20 %<>%
  group_by(CNS_PROF, tempo) %>%
  mutate(VINCULOS = n())

#Descobrindo os hospitais públicos e privados pela natureza juridica
uniao_20 %<>%
  mutate(NAT_JUR = as.character(NAT_JUR),
         PUBLICO = if_else(NAT_JUR == "2011" |
                          NAT_JUR == "Autarquia Federal" |
                          NAT_JUR == "Autarquia Municipal" |
                          NAT_JUR == "Município" |
                          NAT_JUR == "Órgão Público do Poder Executivo Estadual ou do Distrito Federal", 1, 0))

#Transformando subgrupo de profissões em uma geral
uniao_20  %<>%
  mutate(PROFISSAO = case_when(str_detect(PROFISSAO,"Auxiliar de enfermagem") ~ "Técnico de enfermagem",
                               str_starts(PROFISSAO, "Médico") ~ "Médico",
                               str_starts(PROFISSAO, "Fisioterapeuta") ~ "Fisioterapeuta",
                               str_starts(PROFISSAO, "Farmacêutico") ~ "Farmacêutico",
                               str_starts(PROFISSAO, "Enfermeiro") ~ "Enfermeiro",
                               TRUE ~ as.character(PROFISSAO)))

#Número de horas trabalhadas
uniao_20 %<>%
  mutate(HORAOUTR = as.numeric(HORAOUTR),
         HORAHOSP = as.numeric(HORAHOSP),
         HORA_AMB = as.numeric(HORA_AMB),
         H_TOTAL = HORAOUTR + HORAHOSP + HORA_AMB)

#Criando dummies para total de horas trabalhadas:
uniao_20 %<>% mutate(D_HORAS = case_when(H_TOTAL %in% 0:10 ~ 1,
                                         H_TOTAL %in% 11:20 ~ 2,
                                         H_TOTAL %in% 21:30 ~ 3,
                                         H_TOTAL > 30 ~ 4))

#Dummies para indicar o porte do hospital
uniao_20 %<>% mutate(PORTE = case_when(TOTAL <= 50 ~ 1,
                                        TOTAL %in% 51:150 ~ 2,
                                        TOTAL %in% 151:500 ~ 3,
                                        TOTAL > 500 ~ 4))

#Dummies para indicar a quantidade de vínculos trabalhistas
uniao_20  %<>% mutate(D_VINC = case_when(VINCULOS == 1 ~ "1 vínculo",
                                         VINCULOS == 2 ~ "2 vínculos",
                                         VINCULOS >= 3 ~ "3 ou mais vínculos"))

In [61]:
#Unindo a base uniao_19 com codigo para termos as mesorregiões
uniao_20 = inner_join(uniao_20, codigo, by = "CODUFMUN")

In [62]:
#Desagrupando os dados
uniao_20 %<>%
    ungroup()

#### Descrição da base criada

In [63]:
glimpse(uniao_20)

Rows: 43.979
Columns: 24
$ CNES      <chr> "2322153", "2322153", "2322153", "2322153", "2322153", "2322~
$ CNS_PROF  <chr> "207428997690007", "980016288637438", "980016297404556", "98~
$ CODUFMUN  <chr> "250020", "250020", "250020", "250020", "250020", "250020", ~
$ TP_UNID   <chr> "Hospital geral", "Hospital geral", "Hospital geral", "Hospi~
$ CBO       <chr> "225124", "225125", "225125", "223505", "322205", "223605", ~
$ HORAOUTR  <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ~
$ HORAHOSP  <dbl> 0, 0, 12, 0, 0, 0, 12, 40, 0, 40, 0, 0, 12, 0, 0, 0, 0, 0, 0~
$ HORA_AMB  <dbl> 8, 18, 12, 24, 40, 20, 12, 0, 20, 0, 24, 24, 12, 40, 12, 20,~
$ NAT_JUR   <chr> "Órgão Público do Poder Executivo Estadual ou do Distrito Fe~
$ PROFISSAO <chr> "Médico", "Médico", "Médico", "Enfermeiro", "Técnico de enfe~
$ id        <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ~
$ TOTAL     <int> 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, ~
$ t0        <db

#### Exportando os arquivos 

In [64]:
#Arquivo de 2019
write.csv(uniao_19, "uniao_19.csv", row.names = FALSE)

#Arquivo de 2020
write.csv(uniao_20, "uniao_20.csv", row.names = FALSE)
